<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/corpus_maker_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fabrique des Corpus ASAMLA

## Phase 2 : Mise à jour d'un nouveau Corpus
Dans le cas où des modifications, ajouts, suppressions ont fait évoluer le Corpus en français, il faut mettre à jour le Corpus de traduction de travail pour les interprètes

In [ ]:
!export PYTHONIOENCODING=utf8
!pip install gspread-formatting
import pandas as pd
import numpy as np
import base64
import uuid
import zlib
import binascii
import random
import os
import secrets
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_formatting import *
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [ ]:
def make_unique_id():
  return str(uuid.uuid4())[:8]

In [ ]:
def format_feuille(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)
  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:B', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=2)
  set_column_width(ws, 'A', 100)
  set_column_width(ws, 'B', 400)
  set_column_width(ws, 'C:', 600)

Référentiels Corpus de travail

In [ ]:
# Corpus Français Médical Vaccination
corpus_fr_med_vac = "asamla-corpus_francais_med_vac-v1.2"
corpus_fr_med_vac_uri = "https://docs.google.com/spreadsheets/d/1-2SDHCFR5JOHCc4MiYSMo4V8yG7kfHbpIKV3q9ci6wE"
# Corpus Français Médical Neurologie
corpus_fr_med_neu = "asamla-corpus_fr_med_neurologie-v0.1"
corpus_fr_med_neu_uri = "https://docs.google.com/spreadsheets/d/1EmF12up7sCF-KNaEsbGSeL9uco7BUv2enHi4deI12Jk"

In [ ]:
# Corpus Toutes Langues Médical Vaccination
corpus_trad_med_vac = "ASAMLA - Corpus Médical Vaccination-v2.0"
corpus_trad_med_vac_uri = "https://docs.google.com/spreadsheets/d/1CclzYfFCW4srA3Lq_np2LpSrxj84JpcbzytL449DH8E"
# Corpus Toutes Langues Médical Neurologie
corpus_trad_med_neu = "ASAMLA-Corpus Médical Neurologie"
corpus_trad_med_neu_uri = "https://docs.google.com/spreadsheets/d/1RzPCuw1yyfSFPzcttPjXHzMuxSHh4YwpKwgEBS69flU"

Ouverture de la base centrale (repository) : base se trouvant sur gdrive atos.net

In [ ]:
# Ouverture de la Sheet Centrale (corpus_central_base)
sheet_central = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(sheet_central)
# onglet Médical Vaccination
corpus_trad_med_vac_onglet = "med_vac_synthese"
# onglet Médical Neurologie
corpus_trad_med_neu_onglet = "med_neu_synthese"

Chargement du Corpus FR : à ce stade, il ne contient que des expressions en français

In [ ]:
uri = corpus_fr_med_neu_uri

#### 2021 10 05 : Suppression de Peul, remplacé par Allemand

In [ ]:
vk_lang_dict=[
{'language':'Arabe','trigramme':'ams','uri':uri,'idx':1, 'update':'false'},
{'language':'Anglais','trigramme':'eng','uri':uri,'idx':2, 'update':'false'},
{'language':'Turc','trigramme':'tur','uri':uri,'idx':3, 'update':'false'},
{'language':'Russe','trigramme':'rus','uri':uri,'idx':4, 'update':'false'},
{'language':'Ukrainien','trigramme':'ukr','uri':uri,'idx':5, 'update':'false'},
{'language':'Roumain','trigramme':'rou','uri':uri,'idx':6, 'update':'false'},
{'language':'Hongrois','trigramme':'hun','uri':uri,'idx':7, 'update':'false'},
{'language':'Tigrinya','trigramme':'tig','uri':uri,'idx':8, 'update':'false'},
{'language':'Albanais','trigramme':'alb','uri':uri,'idx':9, 'update':'false'},
{'language':'Géorgien','trigramme':'geo','uri':uri,'idx':10, 'update':'false'},
{'language':'Arménien','trigramme':'arm','uri':uri,'idx':11, 'update':'false'},
{'language':'Dari','trigramme':'dar','uri':uri,'idx':12, 'update':'false'},
{'language':'Pashto','trigramme':'pst','uri':uri,'idx':13, 'update':'false'},
{'language':'Fârsi','trigramme':'prs','uri':uri,'idx':14, 'update':'false'},
{'language':'Azéri','trigramme':'aze','uri':uri,'idx':15, 'update':'false'},
{'language':'Espagnol','trigramme':'esp','uri':uri,'idx':16, 'update':'false'},
{'language':'Amharique','trigramme':'amh','uri':uri,'idx':17, 'update':'false'},
{'language':'Allemand','trigramme':'all','uri':uri,'idx':18, 'update':'false'}
]


Fonction d'attribution d'id unique pour chaque expression à l'intérieur d'un corpus

In [ ]:
#ouverture de l'onglet Corpus dans un DF
wb = gc.open_by_url(uri)
t_corpus = wb.get_worksheet(1)
data_t_corpus = t_corpus.get_all_values()
df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])

In [ ]:
# SI L'état de l'expression est VALIDE on ajoute le hash qui sert de clé
df_corpus['uid'] = df_corpus[['uid','état']].apply(lambda x: x[0] if (x[0]!="" and x[1]=="validé") else make_unique_id() if (x[0]=="" and x[1]=="validé") else "", axis=1)
# exemple de bout de code pour accéder à un enregistrement par la clé
# df_corpus.loc[df_corpus['uid']=='g06g6WJ6ROb8-tWX2TlVtwvKY3MdWg']['coded_expression'].apply(lambda x:decode_expression(x))

In [ ]:
# on élimine toutes les lignes sans expression
df_corpus["expression"].replace('', np.nan, inplace=True)
df_corpus = df_corpus[df_corpus['expression'].notna()]
# trim des expressions
df_corpus['français'] = df_corpus['expression'].apply(lambda x:x.strip())

In [ ]:
# on ne conserve que les colonnes utiles
df_corpus = df_corpus[['uid','français']]
# on ajoute la colonne pour la traduction
df_corpus['traduction'] = ''

Sauvegarde du corpus FR avec l'uid

In [ ]:
# sauvegarde du dataframe dans la table wb_central de la feuille gsheet
set_with_dataframe(t_corpus, df_corpus)


In [ ]:
df_corpus = df_corpus.loc[df_corpus['uid']!=""]

#### Mise à jour du corpus français dans un corpus de traduction : 1 langue = 1 onglet

In [ ]:
uri = corpus_trad_med_neu_uri
#ouverture de l'onglet Corpus dans un DF
wb = gc.open_by_url(uri)
#pour chaque corpus, on créé un onglet si besoin
for index, item in enumerate(vk_lang_dict):
  #df_corpus_all.append (df_corpus)
  nom_onglet = item['language']
  # si l'onglet existe déjà, on le supprime ?
  try:
    ws = wb.worksheet(nom_onglet)
    data_onglet = ws.get_all_values()
    df_onglet = pd.DataFrame(data_onglet[1:], columns=data_onglet[0])
    # fusion des données présentes et nouvelles
    frames = [df_onglet, df_corpus]
    result = pd.concat([df_onglet, df_corpus]).drop_duplicates('uid').reset_index(drop=True)
    set_with_dataframe(ws, result)
    format_feuille(wb, nom_onglet)     
  except:
    print ("Onglet inexistant !")
    wb.add_worksheet(nom_onglet, 1, 1)
    export_sheet = wb.worksheet(nom_onglet)
    set_with_dataframe(export_sheet, df_corpus)
    format_feuille(wb, nom_onglet)    

In [ ]:
df_corpus

,uid,français,traduction
0,030d5e5b,Avez-vous des maux de tête ?,
1,3e47a509,Avez-vous mal à la tête ?,
2,f0e17bd2,Avez-vous tendance à avoir des migraines ?,
3,3446fb6e,Prenez-vous des médicaments pour les\r\ncontrô...,
4,0b3b1a57,Céphalée unilatérale,


In [ ]:
df1 = pd.DataFrame([['Apple', 'Jammu & Kashmir'], ['Banana', 'Kerala']],
                    columns=['Fruit', 'State_Grown'])

In [ ]:
df2 = pd.DataFrame([['Apple', ''], ['Watermelon', 'Maharashtra'], ['Banana', ''],['Strawberry', 'Madhya Pradesh']],
                  columns=['Fruit', 'State_Grown'])

In [ ]:
df1

,Fruit,State_Grown
0,Apple,Jammu & Kashmir
1,Banana,Kerala


In [ ]:
df2

,Fruit,State_Grown
0,Apple,
1,Watermelon,Maharashtra
2,Banana,
3,Strawberry,Madhya Pradesh


In [ ]:
pd.concat([df1, df2]).drop_duplicates('Fruit').reset_index(drop=True)

,Fruit,State_Grown
0,Apple,Jammu & Kashmir
1,Banana,Kerala
2,Watermelon,Maharashtra
3,Strawberry,Madhya Pradesh
